In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
# store the dataset.7z file in MiniProject4
sys.path.insert(0,'/content/gdrive/MyDrive/Colab Notebooks/MiniProject4')

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf 
import tensorflow.keras as keras
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import utils_modelnet as ds

# Main Code provided by the paper

## Data loading and pre-processing

### Data Loading

In [ ]:
!7z x '/content/gdrive/MyDrive/Colab Notebooks/MiniProject4/dataset.7z'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/MyDrive/Colab Notebooks/MiniProject4/                                                               1 file, 437541578 bytes (418 MiB)

Extracting archive: /content/gdrive/MyDrive/Colab Notebooks/MiniProject4/dataset.7z
--
Path = /content/gdrive/MyDrive/Colab Notebooks/MiniProject4/dataset.7z
Type = 7z
Physical Size = 437541578
Headers Size = 459592
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 84         0% 84 - dataset/modelnet2d/airplane/1/1.png                                               

### Pre-processing

In [ ]:
dataset_path = 'dataset/modelnet2d/'
class_set =  ['chair', 'car', 'lamp', 'airplane', 'person']

dataset = ds.get_data_from_file(class_set, dataset_path)
train_dataset, valid_dataset, test_dataset = ds.train_test_split(dataset)

train_data, train_label = ds.split_data_label(train_dataset)
test_data, test_label = ds.split_data_label(test_dataset)
valid_data, valid_label = ds.split_data_label(valid_dataset)
## train and validation
print("Train Dataset: {}".format(len(train_dataset)))
print("Test Dataset: {}".format(len(test_dataset)))
print("Valid Dataset: {}".format(len(valid_dataset)))
num_classes = len(class_set)
print("Number of Classes: {}".format(num_classes))
BATCH_SIZE = 32
IMG_SIZE = 48
NUM_CHANNEL = 1

Train Dataset: 10800
Test Dataset: 32400
Valid Dataset: 5400
Number of Classes: 5


### Preparing binocular images

In [ ]:
def get_binocular_dataset(data, label, batch_size=BATCH_SIZE):
    def preprocess_image(left_image, right_image):
        left_image = tf.image.decode_jpeg(left_image, channels=NUM_CHANNEL)
        left_image = tf.image.resize(left_image, [IMG_SIZE, IMG_SIZE])
        left_image /= 255.0

        right_image = tf.image.decode_jpeg(right_image, channels=NUM_CHANNEL)
        right_image = tf.image.resize(right_image, [IMG_SIZE, IMG_SIZE])
        right_image /= 255.0  # normalize to [0,1] range
        return left_image, right_image

    def load_and_preprocess_image(left, right):
        left_image = tf.io.read_file(left)
        right_image = tf.io.read_file(right)
        return preprocess_image(left_image, right_image)

    # The tuples are unpacked into the positional arguments of the mapped function 
    def load_and_preprocess_from_path_label(data_path, label):
        return load_and_preprocess_image(data_path[0], data_path[1]), label
    
    ds = tf.data.Dataset.from_tensor_slices((data, label))
    ds = ds.map(load_and_preprocess_from_path_label, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.shuffle(buffer_size=len(data))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    print(ds)
    return ds

In [ ]:
train_ds = get_binocular_dataset(train_data, train_label)
test_ds = get_binocular_dataset(test_data, test_label)
valid_ds = get_binocular_dataset(valid_data, valid_label)

<PrefetchDataset shapes: (((None, 48, 48, 1), (None, 48, 48, 1)), (None,)), types: ((tf.float32, tf.float32), tf.int32)>
<PrefetchDataset shapes: (((None, 48, 48, 1), (None, 48, 48, 1)), (None,)), types: ((tf.float32, tf.float32), tf.int32)>
<PrefetchDataset shapes: (((None, 48, 48, 1), (None, 48, 48, 1)), (None,)), types: ((tf.float32, tf.float32), tf.int32)>


## CNN2

### Defining Monocular layer including cmpooling and Convolution

In [ ]:
## Utility function
def cmpooling(fmaps, scale_list, pool_stride):
    # make sure the scale_list is in decending order
    if scale_list[0] - scale_list[1] < 0:
        scale_list = scale_list[::-1]
        
    # concentric multi-scale pooling
    offset = [0] + [-(scale_list[i+1] - scale_list[0])//2 for i in range(len(scale_list) - 1)]
    pool_maps = []
    for offset, scale in zip(offset, scale_list):
        slice_maps = tf.slice(fmaps, [0, offset, offset, 0], [-1, fmaps.shape[1]-offset*2, fmaps.shape[2]-offset*2, -1])
        pool_map = tf.nn.max_pool2d(slice_maps, scale, pool_stride, "VALID")
        pool_maps.append(pool_map)
    
    # assert same shape for all pool_map
    for i in range(len(pool_maps)-1):
        assert pool_maps[i].shape[1:] == pool_maps[-1].shape[1:]
    return pool_maps

# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        pool_maps = cmpooling(fmaps, scale_list, pool_stride)
        pool_maps = tf.concat(pool_maps, axis=-1)
        return self.conv(pool_maps)

### Defining CNN2 Model

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)
    right = tf.concat([right_eye, parallax], axis=-1)
    # 
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)
    
    left2 = Monocular(12, 5, name='mono2_left')(tf.concat([left1, right1], axis=-1), scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(tf.concat([right1, left1], axis=-1), scale_list=scale_list, pool_stride=1)
    
    left3 = Monocular(32, 3, name='mono3_left')(tf.concat([left2, right2], axis=-1), scale_list=scale_list, pool_stride=1)
    right3 = Monocular(32, 3, name='mono3_right')(tf.concat([right2, left2], axis=-1), scale_list=scale_list, pool_stride=1)
    
    x = tf.concat([left3, right3], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
def create_model(model, input_shape, num_classes, scale_list):
    m = model(input_shape, num_classes, scale_list)
    # learning rate schedule
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000,
                                                              decay_rate=0.96, staircase=True)
    
    # compile the model
    m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # Optimizer
                  # Loss function to minimize
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  # List of metrics to monitor
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                 )
    return m

### Create and fitting the model

In [ ]:
cnn2 = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2.fit(train_ds , epochs=10, validation_data=valid_ds)


Epoch 1/10
338/338 [==============================] - 19s 26ms/step - loss: 0.7943 - sparse_categorical_accuracy: 0.7154 - val_loss: 0.1203 - val_sparse_categorical_accuracy: 0.9630
Epoch 2/10
338/338 [==============================] - 18s 25ms/step - loss: 0.1168 - sparse_categorical_accuracy: 0.9625 - val_loss: 0.0880 - val_sparse_categorical_accuracy: 0.9713
Epoch 3/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0634 - sparse_categorical_accuracy: 0.9812 - val_loss: 0.0873 - val_sparse_categorical_accuracy: 0.9719
Epoch 4/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0477 - sparse_categorical_accuracy: 0.9849 - val_loss: 0.0538 - val_sparse_categorical_accuracy: 0.9820
Epoch 5/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0440 - sparse_categorical_accuracy: 0.9867 - val_loss: 0.0349 - val_sparse_categorical_accuracy: 0.9874
Epoch 6/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0364 - spars

### Evaluation on test dataset

In [ ]:
loss, acc = cnn2.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 34s 4ms/step - loss: 0.0180 - sparse_categorical_accuracy: 0.9940
Loss: 0.018022213131189346, Acc: 0.9939814805984497


# Experiments

## Using different number of scales in CMPooling (additional to the paper experiments)

We changed the scal_list variable to [1,3] and [1,3,5,7] to see the effect of using less and more scales on the accuracy.

### Using 2 scales: 1, 3

In [ ]:
SCALE_LIST = [1,3]
cnn2_2scales = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_2scales.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 20s 28ms/step - loss: 0.7514 - sparse_categorical_accuracy: 0.7360 - val_loss: 0.1125 - val_sparse_categorical_accuracy: 0.9659
Epoch 2/10
338/338 [==============================] - 19s 27ms/step - loss: 0.1212 - sparse_categorical_accuracy: 0.9587 - val_loss: 0.0826 - val_sparse_categorical_accuracy: 0.9731
Epoch 3/10
338/338 [==============================] - 19s 27ms/step - loss: 0.0552 - sparse_categorical_accuracy: 0.9822 - val_loss: 0.0349 - val_sparse_categorical_accuracy: 0.9891
Epoch 4/10
338/338 [==============================] - 19s 27ms/step - loss: 0.0275 - sparse_categorical_accuracy: 0.9920 - val_loss: 0.0279 - val_sparse_categorical_accuracy: 0.9919
Epoch 5/10
338/338 [==============================] - 19s 27ms/step - loss: 0.0246 - sparse_categorical_accuracy: 0.9919 - val_loss: 0.0315 - val_sparse_categorical_accuracy: 0.9909
Epoch 6/10
338/338 [==============================] - 20s 29ms/step - loss: 0.0197 - spars

In [ ]:
loss, acc = cnn2_2scales.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 34s 4ms/step - loss: 0.0409 - sparse_categorical_accuracy: 0.9865
Loss: 0.04089761897921562, Acc: 0.9865123629570007


### Using 4 scales: 1, 3, 5, 7

In [ ]:
SCALE_LIST = [1,3,5,7]
cnn2_4scales = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_4scales.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 19s 24ms/step - loss: 0.9522 - sparse_categorical_accuracy: 0.6308 - val_loss: 0.1602 - val_sparse_categorical_accuracy: 0.9507
Epoch 2/10
338/338 [==============================] - 18s 23ms/step - loss: 0.1453 - sparse_categorical_accuracy: 0.9551 - val_loss: 0.0982 - val_sparse_categorical_accuracy: 0.9730
Epoch 3/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0861 - sparse_categorical_accuracy: 0.9731 - val_loss: 0.0916 - val_sparse_categorical_accuracy: 0.9654
Epoch 4/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0675 - sparse_categorical_accuracy: 0.9787 - val_loss: 0.0559 - val_sparse_categorical_accuracy: 0.9811
Epoch 5/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0515 - sparse_categorical_accuracy: 0.9859 - val_loss: 0.0950 - val_sparse_categorical_accuracy: 0.9648
Epoch 6/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0453 - spars

In [ ]:
loss, acc = cnn2_4scales.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 33s 4ms/step - loss: 0.0288 - sparse_categorical_accuracy: 0.9905
Loss: 0.02878613770008087, Acc: 0.9905247092247009


We can see the accuracy when we use [1,3,5] as scale list is better than the other two.

## Pooling after Convolution

investigating the effect of placing cmpooling after conv layer on the accuracy

### Updated Monocular layer for Pooling after Convolution

In [ ]:
# Changing SCALE_LIST to the original one
SCALE_LIST = [1,3,5]
# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        conv_maps = self.conv(fmaps)
        pool_maps = cmpooling(conv_maps, scale_list, pool_stride)
        return tf.concat(pool_maps, axis=-1)

### Create and fitting the model

In [ ]:
cnn2_inverseP = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_inverseP.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 21s 28ms/step - loss: 0.8101 - sparse_categorical_accuracy: 0.7082 - val_loss: 0.1007 - val_sparse_categorical_accuracy: 0.9626
Epoch 2/10
338/338 [==============================] - 19s 28ms/step - loss: 0.0832 - sparse_categorical_accuracy: 0.9727 - val_loss: 0.0487 - val_sparse_categorical_accuracy: 0.9831
Epoch 3/10
338/338 [==============================] - 20s 28ms/step - loss: 0.0378 - sparse_categorical_accuracy: 0.9882 - val_loss: 0.0536 - val_sparse_categorical_accuracy: 0.9813
Epoch 4/10
338/338 [==============================] - 19s 28ms/step - loss: 0.0282 - sparse_categorical_accuracy: 0.9917 - val_loss: 0.0252 - val_sparse_categorical_accuracy: 0.9920
Epoch 5/10
338/338 [==============================] - 19s 27ms/step - loss: 0.0181 - sparse_categorical_accuracy: 0.9938 - val_loss: 0.0394 - val_sparse_categorical_accuracy: 0.9874
Epoch 6/10
338/338 [==============================] - 19s 28ms/step - loss: 0.0164 - spars

### Evaluation on test dataset

In [ ]:
loss, acc = cnn2_inverseP.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 34s 5ms/step - loss: 0.0134 - sparse_categorical_accuracy: 0.9960
Loss: 0.013400280848145485, Acc: 0.9959567785263062


## Ablation study

### 1) Concentric multi-scale (CM) pooling contribution

Using Maxpooling instead of CMPooling

In [ ]:
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        pool_maps = tf.keras.layers.MaxPooling2D((2, 2), pool_stride, "SAME")(fmaps)
        return self.conv(pool_maps)

In [ ]:
cnn2_maxPool = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_maxPool.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 20s 28ms/step - loss: 0.8976 - sparse_categorical_accuracy: 0.6637 - val_loss: 0.1714 - val_sparse_categorical_accuracy: 0.9450
Epoch 2/10
338/338 [==============================] - 19s 27ms/step - loss: 0.1406 - sparse_categorical_accuracy: 0.9566 - val_loss: 0.0945 - val_sparse_categorical_accuracy: 0.9661
Epoch 3/10
338/338 [==============================] - 19s 28ms/step - loss: 0.0788 - sparse_categorical_accuracy: 0.9724 - val_loss: 0.0482 - val_sparse_categorical_accuracy: 0.9863
Epoch 4/10
338/338 [==============================] - 19s 28ms/step - loss: 0.0543 - sparse_categorical_accuracy: 0.9818 - val_loss: 0.0424 - val_sparse_categorical_accuracy: 0.9889
Epoch 5/10
338/338 [==============================] - 19s 27ms/step - loss: 0.0323 - sparse_categorical_accuracy: 0.9901 - val_loss: 0.0308 - val_sparse_categorical_accuracy: 0.9920
Epoch 6/10
338/338 [==============================] - 19s 27ms/step - loss: 0.0235 - spars

In [ ]:
loss, acc = cnn2_maxPool.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 34s 5ms/step - loss: 0.0232 - sparse_categorical_accuracy: 0.9938
Loss: 0.02324630320072174, Acc: 0.9937654137611389


### 2) Changing MaxPooling to AvgPooling in CMPooling layer (additional to the paper experiments)

In [ ]:
# Changing the Monocular to the original one
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        pool_maps = cmpooling(fmaps, scale_list, pool_stride)
        pool_maps = tf.concat(pool_maps, axis=-1)
        return self.conv(pool_maps)

In [ ]:
## Utility function
def cmpooling(fmaps, scale_list, pool_stride):
    # make sure the scale_list is in decending order
    if scale_list[0] - scale_list[1] < 0:
        scale_list = scale_list[::-1]
        
    # concentric multi-scale pooling
    offset = [0] + [-(scale_list[i+1] - scale_list[0])//2 for i in range(len(scale_list) - 1)]
    pool_maps = []
    for offset, scale in zip(offset, scale_list):
        slice_maps = tf.slice(fmaps, [0, offset, offset, 0], [-1, fmaps.shape[1]-offset*2, fmaps.shape[2]-offset*2, -1])
        pool_map = tf.nn.avg_pool2d(slice_maps, scale, pool_stride, "VALID")
        pool_maps.append(pool_map)
    
    # assert same shape for all pool_map
    for i in range(len(pool_maps)-1):
        assert pool_maps[i].shape[1:] == pool_maps[-1].shape[1:]
    return pool_maps

In [ ]:
cnn2_CM_avgPooling = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_CM_avgPooling.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 19s 25ms/step - loss: 0.7789 - sparse_categorical_accuracy: 0.7197 - val_loss: 0.1299 - val_sparse_categorical_accuracy: 0.9537
Epoch 2/10
338/338 [==============================] - 18s 25ms/step - loss: 0.1273 - sparse_categorical_accuracy: 0.9546 - val_loss: 0.0767 - val_sparse_categorical_accuracy: 0.9719
Epoch 3/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0826 - sparse_categorical_accuracy: 0.9694 - val_loss: 0.1188 - val_sparse_categorical_accuracy: 0.9543
Epoch 4/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0775 - sparse_categorical_accuracy: 0.9710 - val_loss: 0.0440 - val_sparse_categorical_accuracy: 0.9878
Epoch 5/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0361 - sparse_categorical_accuracy: 0.9867 - val_loss: 0.0444 - val_sparse_categorical_accuracy: 0.9843
Epoch 6/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0339 - spars

In [ ]:
loss, acc = cnn2.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 33s 4ms/step - loss: 0.0186 - sparse_categorical_accuracy: 0.9940
Loss: 0.01860054023563862, Acc: 0.993950605392456


### 3) One feedforward pathways

In [ ]:
# Changing the cmpooling to the original one
## Utility function
def cmpooling(fmaps, scale_list, pool_stride):
    # make sure the scale_list is in decending order
    if scale_list[0] - scale_list[1] < 0:
        scale_list = scale_list[::-1]
        
    # concentric multi-scale pooling
    offset = [0] + [-(scale_list[i+1] - scale_list[0])//2 for i in range(len(scale_list) - 1)]
    pool_maps = []
    for offset, scale in zip(offset, scale_list):
        slice_maps = tf.slice(fmaps, [0, offset, offset, 0], [-1, fmaps.shape[1]-offset*2, fmaps.shape[2]-offset*2, -1])
        pool_map = tf.nn.max_pool2d(slice_maps, scale, pool_stride, "VALID")
        pool_maps.append(pool_map)
    
    # assert same shape for all pool_map
    for i in range(len(pool_maps)-1):
        assert pool_maps[i].shape[1:] == pool_maps[-1].shape[1:]
    return pool_maps

In [ ]:
# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        conv_maps = self.conv(fmaps)
        pool_maps = tf.keras.layers.MaxPooling2D((2, 2), pool_stride, "SAME")(conv_maps)
        
        return pool_maps

In [ ]:
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    x = tf.concat([left_eye, right_eye], axis=-1)

    x = Monocular(6, 5, input_shape=input_shape, name='mono1')(x, scale_list=scale_list, pool_stride=2)    
    x = Monocular(12, 5, name='mono2')(x, scale_list=scale_list, pool_stride=1)    
    x = Monocular(32, 3, name='mono3')(x, scale_list=scale_list, pool_stride=1)
    
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_1fP = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_1fP.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 19s 27ms/step - loss: 0.9160 - sparse_categorical_accuracy: 0.6652 - val_loss: 0.1371 - val_sparse_categorical_accuracy: 0.9563
Epoch 2/10
338/338 [==============================] - 19s 26ms/step - loss: 0.1243 - sparse_categorical_accuracy: 0.9608 - val_loss: 0.1572 - val_sparse_categorical_accuracy: 0.9426
Epoch 3/10
338/338 [==============================] - 19s 26ms/step - loss: 0.0603 - sparse_categorical_accuracy: 0.9804 - val_loss: 0.0752 - val_sparse_categorical_accuracy: 0.9767
Epoch 4/10
338/338 [==============================] - 19s 26ms/step - loss: 0.0521 - sparse_categorical_accuracy: 0.9840 - val_loss: 0.0519 - val_sparse_categorical_accuracy: 0.9822
Epoch 5/10
338/338 [==============================] - 19s 26ms/step - loss: 0.0311 - sparse_categorical_accuracy: 0.9912 - val_loss: 0.0273 - val_sparse_categorical_accuracy: 0.9907
Epoch 6/10
338/338 [==============================] - 19s 26ms/step - loss: 0.0172 - spars

In [ ]:
loss, acc = cnn2_1fP.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 34s 4ms/step - loss: 0.0218 - sparse_categorical_accuracy: 0.9942
Loss: 0.021849771961569786, Acc: 0.9941975474357605


### 4) Changing number of (CMPooling + conv) layers (additional to the paper experiments)

In [ ]:
# Changing the Monocular to the original one
# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        pool_maps = cmpooling(fmaps, scale_list, pool_stride)
        pool_maps = tf.concat(pool_maps, axis=-1)
        return self.conv(pool_maps)

2 (CMPooling + conv) layers 

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)
    right = tf.concat([right_eye, parallax], axis=-1)
    # 
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)
    
    left2 = Monocular(12, 5, name='mono2_left')(tf.concat([left1, right1], axis=-1), scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(tf.concat([right1, left1], axis=-1), scale_list=scale_list, pool_stride=1)
    
    x = tf.concat([left2, right2], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_2cm_conv_layers = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_2cm_conv_layers.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 19s 25ms/step - loss: 0.8166 - sparse_categorical_accuracy: 0.6949 - val_loss: 0.1515 - val_sparse_categorical_accuracy: 0.9467
Epoch 2/10
338/338 [==============================] - 18s 24ms/step - loss: 0.1287 - sparse_categorical_accuracy: 0.9595 - val_loss: 0.0969 - val_sparse_categorical_accuracy: 0.9665
Epoch 3/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0890 - sparse_categorical_accuracy: 0.9707 - val_loss: 0.0564 - val_sparse_categorical_accuracy: 0.9826
Epoch 4/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0582 - sparse_categorical_accuracy: 0.9807 - val_loss: 0.0576 - val_sparse_categorical_accuracy: 0.9804
Epoch 5/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0364 - sparse_categorical_accuracy: 0.9905 - val_loss: 0.0523 - val_sparse_categorical_accuracy: 0.9839
Epoch 6/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0403 - spars

In [ ]:
loss, acc = cnn2_2cm_conv_layers.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 33s 4ms/step - loss: 0.0217 - sparse_categorical_accuracy: 0.9934
Loss: 0.021721355617046356, Acc: 0.9933642148971558


4 (CMPooling + conv) layers 

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)
    right = tf.concat([right_eye, parallax], axis=-1)
    # 
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)
    
    left2 = Monocular(12, 5, name='mono2_left')(tf.concat([left1, right1], axis=-1), scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(tf.concat([right1, left1], axis=-1), scale_list=scale_list, pool_stride=1)
    
    left3 = Monocular(32, 3, name='mono3_left')(tf.concat([left2, right2], axis=-1), scale_list=scale_list, pool_stride=1)
    right3 = Monocular(32, 3, name='mono3_right')(tf.concat([right2, left2], axis=-1), scale_list=scale_list, pool_stride=1)
    
    left4 = Monocular(64, 3, name='mono4_left')(tf.concat([left3, right3], axis=-1), scale_list=scale_list, pool_stride=1)
    right4 = Monocular(64, 3, name='mono4_right')(tf.concat([right3, left3], axis=-1), scale_list=scale_list, pool_stride=1)

    x = tf.concat([left4, right4], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_4cm_conv_layers = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_4cm_conv_layers.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 20s 26ms/step - loss: 0.7916 - sparse_categorical_accuracy: 0.7294 - val_loss: 0.1402 - val_sparse_categorical_accuracy: 0.9494
Epoch 2/10
338/338 [==============================] - 18s 25ms/step - loss: 0.1573 - sparse_categorical_accuracy: 0.9469 - val_loss: 0.0911 - val_sparse_categorical_accuracy: 0.9726
Epoch 3/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0824 - sparse_categorical_accuracy: 0.9716 - val_loss: 0.0606 - val_sparse_categorical_accuracy: 0.9811
Epoch 4/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0646 - sparse_categorical_accuracy: 0.9792 - val_loss: 0.0473 - val_sparse_categorical_accuracy: 0.9852
Epoch 5/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0588 - sparse_categorical_accuracy: 0.9803 - val_loss: 0.0653 - val_sparse_categorical_accuracy: 0.9743
Epoch 6/10
338/338 [==============================] - 18s 26ms/step - loss: 0.0406 - spars

In [ ]:
loss, acc = cnn2_4cm_conv_layers.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 33s 4ms/step - loss: 0.0217 - sparse_categorical_accuracy: 0.9925
Loss: 0.02171819470822811, Acc: 0.9925000071525574


### 5) Without augmentation

CNN2 without Augmentation

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    left = left_eye
    right = right_eye
    # 
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)
    
    left2 = Monocular(12, 5, name='mono2_left')(tf.concat([left1, right1], axis=-1), scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(tf.concat([right1, left1], axis=-1), scale_list=scale_list, pool_stride=1)
    
    left3 = Monocular(32, 3, name='mono3_left')(tf.concat([left2, right2], axis=-1), scale_list=scale_list, pool_stride=1)
    right3 = Monocular(32, 3, name='mono3_right')(tf.concat([right2, left2], axis=-1), scale_list=scale_list, pool_stride=1)
    
    x = tf.concat([left3, right3], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_WoAug = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_WoAug.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 19s 26ms/step - loss: 0.8668 - sparse_categorical_accuracy: 0.7021 - val_loss: 0.2407 - val_sparse_categorical_accuracy: 0.9107
Epoch 2/10
338/338 [==============================] - 18s 25ms/step - loss: 0.1457 - sparse_categorical_accuracy: 0.9460 - val_loss: 0.1491 - val_sparse_categorical_accuracy: 0.9448
Epoch 3/10
338/338 [==============================] - 18s 25ms/step - loss: 0.1061 - sparse_categorical_accuracy: 0.9645 - val_loss: 0.0754 - val_sparse_categorical_accuracy: 0.9726
Epoch 4/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0561 - sparse_categorical_accuracy: 0.9795 - val_loss: 0.0610 - val_sparse_categorical_accuracy: 0.9800
Epoch 5/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0454 - sparse_categorical_accuracy: 0.9864 - val_loss: 0.0369 - val_sparse_categorical_accuracy: 0.9874
Epoch 6/10
338/338 [==============================] - 18s 25ms/step - loss: 0.0361 - spars

In [ ]:
loss, acc = cnn2_WoAug.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 34s 4ms/step - loss: 0.0133 - sparse_categorical_accuracy: 0.9958
Loss: 0.01327881682664156, Acc: 0.995771586894989


## Fusion of the Two Feedforward Pathways

In [ ]:
SCALE_LIST = [1,3,5]
def CNN2(input_shape, num_classes, scale_list):
    left_eye = tf.keras.Input(input_shape, name='left_eye')
    right_eye = tf.keras.Input(input_shape, name='right_eye')
    
    # parallax augmentation
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)
    right = tf.concat([right_eye, parallax], axis=-1)
    # 
    
    left1 = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    right1 = Monocular(6, 5, input_shape=input_shape, name='mono1_right')(right, scale_list=scale_list, pool_stride=2)

    left2 = Monocular(12, 5, name='mono2_left')(left1, scale_list=scale_list, pool_stride=1)
    right2 = Monocular(12, 5, name='mono2_right')(right1, scale_list=scale_list, pool_stride=1)

    left3 = Monocular(32, 3, name='mono3_left')(left2, scale_list=scale_list, pool_stride=1)
    right3 = Monocular(32, 3, name='mono3_right')(right2, scale_list=scale_list, pool_stride=1)
    
    x = tf.concat([left3, right3], axis=-1)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(feature_vector)
    
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
cnn2_fu = create_model(CNN2, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn2_fu.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 19s 25ms/step - loss: 1.0622 - sparse_categorical_accuracy: 0.5998 - val_loss: 0.2013 - val_sparse_categorical_accuracy: 0.9333
Epoch 2/10
338/338 [==============================] - 18s 25ms/step - loss: 0.1633 - sparse_categorical_accuracy: 0.9494 - val_loss: 0.1140 - val_sparse_categorical_accuracy: 0.9617
Epoch 3/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0933 - sparse_categorical_accuracy: 0.9686 - val_loss: 0.0934 - val_sparse_categorical_accuracy: 0.9665
Epoch 4/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0593 - sparse_categorical_accuracy: 0.9795 - val_loss: 0.0841 - val_sparse_categorical_accuracy: 0.9689
Epoch 5/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0434 - sparse_categorical_accuracy: 0.9854 - val_loss: 0.0349 - val_sparse_categorical_accuracy: 0.9878
Epoch 6/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0332 - spars

In [ ]:
loss, acc = cnn2_fu.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 34s 4ms/step - loss: 0.0193 - sparse_categorical_accuracy: 0.9928
Loss: 0.019315484911203384, Acc: 0.9928086400032043


# Vanilla CNN

In [ ]:
def CNN(input_shape, num_classes):
    
    left_eye= tf.keras.Input(shape=input_shape, name = 'left_eye')
    right_eye= tf.keras.Input(shape=input_shape, name = 'right_eye')
    
    parallax = left_eye - right_eye 
    x = tf.concat([left_eye, -parallax], axis=-1)
    
    x = tf.keras.layers.Conv2D(6, 5,  activation='relu', padding= 'same', name = 'mono1_left')(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(12, 5, activation='relu', padding= 'same', name='mono2_left')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,1))(x)
    x = tf.keras.layers.Conv2D(32, 3, activation='relu', padding= 'same', name='mono3_left')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,1))(x)
    x = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(x)
    x = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(x)
    x = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(x)
    feature_vector = tf.keras.layers.Flatten()(x)
    predicted_output = tf.keras.layers.Dense(num_classes, activation="softmax", name='output')(feature_vector)
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
def create_model(model, input_shape, num_classes):
    m = model(input_shape, num_classes)
    # learning rate schedule
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000,
                                                              decay_rate=0.96, staircase=True)
    
    # compile the model
    m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # Optimizer
                  # Loss function to minimize
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  # List of metrics to monitor
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                 )
    return m

In [ ]:
cnn = create_model(CNN, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5)
cnn.fit(train_ds , epochs=10, validation_data=valid_ds)

Epoch 1/10
338/338 [==============================] - 19s 26ms/step - loss: 0.9657 - sparse_categorical_accuracy: 0.6468 - val_loss: 0.1684 - val_sparse_categorical_accuracy: 0.9389
Epoch 2/10
338/338 [==============================] - 19s 26ms/step - loss: 0.1368 - sparse_categorical_accuracy: 0.9522 - val_loss: 0.0637 - val_sparse_categorical_accuracy: 0.9796
Epoch 3/10
338/338 [==============================] - 19s 26ms/step - loss: 0.0560 - sparse_categorical_accuracy: 0.9842 - val_loss: 0.0442 - val_sparse_categorical_accuracy: 0.9852
Epoch 4/10
338/338 [==============================] - 18s 26ms/step - loss: 0.0442 - sparse_categorical_accuracy: 0.9865 - val_loss: 0.0310 - val_sparse_categorical_accuracy: 0.9915
Epoch 5/10
338/338 [==============================] - 19s 26ms/step - loss: 0.0306 - sparse_categorical_accuracy: 0.9906 - val_loss: 0.0383 - val_sparse_categorical_accuracy: 0.9881
Epoch 6/10
338/338 [==============================] - 18s 26ms/step - loss: 0.0162 - spars

In [ ]:
loss, acc = cnn.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 34s 4ms/step - loss: 0.0147 - sparse_categorical_accuracy: 0.9956
Loss: 0.01468063984066248, Acc: 0.995555579662323


## VanillaCNN with CMPooling

In [ ]:
# Concat the feature maps in different scale and convolution once. (paper version)
class Monocular(tf.keras.layers.Layer):
    def __init__(self, filters, ksize, **kwargs):
        super(Monocular, self).__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize
#         self.conv = tf.keras.layers.Conv2D(filters, ksize, input_shape=kwargs['input_shape'], activation='relu', padding='same')
    
    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.filters, self.ksize, input_shape=input_shape, activation='relu', padding='same')
    
    def call(self, fmaps, scale_list, pool_stride):
        conv_maps = self.conv(fmaps)
        pool_maps = cmpooling(conv_maps, scale_list, pool_stride)
        return tf.concat(pool_maps, axis=-1)

In [ ]:
SCALE_LIST = [1,3,5]
def CNN(input_shape, num_classes, scale_list):
    
    left_eye= tf.keras.Input(shape=input_shape, name = 'left_eye')
    right_eye= tf.keras.Input(shape=input_shape, name = 'right_eye')
    
    parallax = left_eye - right_eye 
    left = tf.concat([left_eye, -parallax], axis=-1)

    y = Monocular(6, 5, input_shape=input_shape, name='mono1_left')(left, scale_list=scale_list, pool_stride=2)
    y = Monocular(12, 5, input_shape=input_shape, name='mono2_left')(y, scale_list=scale_list, pool_stride=1)
    y = Monocular(32, 3, input_shape=input_shape, name='mono3_left')(y, scale_list=scale_list, pool_stride=1)
    
    y = tf.keras.layers.Conv2D(256, 3, strides=1, activation='relu', name='conv1')(y)
    y = tf.keras.layers.Conv2D(256, 1, strides=1, activation='relu', name='conv2')(y)
    y = tf.keras.layers.Conv2D(64, 1, strides=1, activation='relu', name='conv3')(y)
    feature_vector = tf.keras.layers.Flatten()(y)
    predicted_output = tf.keras.layers.Dense(num_classes, activation="softmax", name='output')(feature_vector)
    return tf.keras.Model([left_eye, right_eye], predicted_output)

In [ ]:
def create_model(model, input_shape, num_classes, scale_list):
    m = model(input_shape, num_classes, scale_list)
    # learning rate schedule
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000,
                                                              decay_rate=0.96, staircase=True)
    
    # compile the model
    m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # Optimizer
                  # Loss function to minimize
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  # List of metrics to monitor
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                 )
    return m

In [ ]:
cnn_cmp = create_model(CNN, input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNEL), num_classes=5, scale_list=SCALE_LIST)
cnn_cmp.fit(train_ds , epochs=10, validation_data=valid_ds, shuffle=False)

Epoch 1/10
338/338 [==============================] - 19s 25ms/step - loss: 0.8702 - sparse_categorical_accuracy: 0.6758 - val_loss: 0.1958 - val_sparse_categorical_accuracy: 0.9291
Epoch 2/10
338/338 [==============================] - 18s 24ms/step - loss: 0.1288 - sparse_categorical_accuracy: 0.9562 - val_loss: 0.0778 - val_sparse_categorical_accuracy: 0.9759
Epoch 3/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0649 - sparse_categorical_accuracy: 0.9800 - val_loss: 0.0505 - val_sparse_categorical_accuracy: 0.9848
Epoch 4/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0511 - sparse_categorical_accuracy: 0.9841 - val_loss: 0.0491 - val_sparse_categorical_accuracy: 0.9852
Epoch 5/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0376 - sparse_categorical_accuracy: 0.9879 - val_loss: 0.0390 - val_sparse_categorical_accuracy: 0.9863
Epoch 6/10
338/338 [==============================] - 18s 24ms/step - loss: 0.0287 - spars

In [ ]:
loss, acc = cnn_cmp.evaluate(test_ds)
print(f"Loss: {loss}, Acc: {acc}")

1013/1013 [==============================] - 33s 4ms/step - loss: 0.0440 - sparse_categorical_accuracy: 0.9852
Loss: 0.04395705461502075, Acc: 0.9852468967437744
